In [1]:
import tweepy
import csv
import json
from config import (consumer_key, consumer_secret,
                    access_token, access_token_secret)
import wget
from bson import ObjectId # For ObjectId to work
from pymongo import MongoClient
import pandas as pd
from datetime import datetime
import numpy as np
import time


In [2]:
uri = "mongodb://newsdata:ZG4cTh2XbyUpkiVggEcQ6lL6UEC32iJThy4eZrO9RLY1suKiXdUNrqi5pzrqzqNvUsdO9xAxjhyFfNfNVoBeXA==@newsdata.documents.azure.com:10255/?ssl=true&replicaSet=globaldb"
client = MongoClient(uri)
db = client.fakenewsdb
politico_facts_data_table = db.politico_facts_data
print(politico_facts_data_table.find_one())

{'_id': ObjectId('5c23d50044925f3eb8d1a2ef'), 'Date': 'on Sunday, December 23rd, 2018', 'Meter': 'Half-True', 'Source': 'Garry McCarthy', 'Statement': '"There was almost a 70 percent reduction in police-related shootings under my tenure because of training, supervision and policy decisions."\r\n', 'Statement_url': '/illinois/statements/2018/dec/23/garry-mccarthy/did-police-shootings-plunge-under-mccarthys-leader/', 'Fact': "The numbers fell, but that doesn't mean he can take all the credit", 'Source_Url': '/personalities/garry-mccarthy/statements/byruling/half-true/'}


In [3]:
fakenews_datatable = db.fakenews_data
print(fakenews_datatable.find_one())

{'_id': ObjectId('5c3445ffe8a490580c567e3c'), 'id': 'politifact14927', 'text_content': {'movies': [], 'images': ['https://ichef.bbci.co.uk/news/1024/media/images/71100000/jpg/_71100784_5c13818a-b5e6-4be8-a7dd-fc65d00632d7.jpg', 'http://ssc.api.bbc.com/?c1=2&c2=19293874&ns_site=bbc&name=news.av.entertainment-arts-24934979.oprah-winfrey-the-butler-racism-and-obama', 'https://news.bbcimg.co.uk/media/images/65840000/jpg/_65840685_hi017217364.jpg', 'https://news.bbcimg.co.uk/media/images/69254000/jpg/_69254318_69254317.jpg', 'https://news.bbcimg.co.uk/media/images/69265000/jpg/_69265117_69265116.jpg', 'https://ichef.bbci.co.uk/news/640/media/images/71100000/jpg/_71100780_71100779.jpg', 'https://sa.bbc.co.uk/bbc/bbc/s?name=news.entertainment_and_arts.media_asset.24934979.page&ml_name=webmodule&ml_version=95&blq_js_enabled=0&blq_s=4d&blq_r=2.7&blq_v=default&blq_e=pal&cps_asset_id=24934979&page_type=media_asset&section=%2Fnews%2Fentertainment_and_arts&first_pub=2013-11-13T18%3A25%3A48%2B00%3A0

In [18]:
tweet = fakenews_datatable.find_one()['tweets']

In [19]:
tweet

[{'reply': [],
  'text': 'Breaking News - VIDEO: Winfrey: The Butler, racism and Obama: American broadcaster and actress O... http://bbc.in/1cp6nkD  - BBCBreaking',
  'created_at': 1384377137,
  'retweet': [],
  'user_name': 'AfricainLive',
  'likes': [],
  'tweet_id': 400732875898638336,
  'user_id': 1308616477},
 {'reply': [],
  'text': 'BBC News: VIDEO: Winfrey: The Butler, racism and Obama: American broadcaster and actress Oprah Winfrey sai... http://bbc.in/1cp6pZW  #fb',
  'created_at': 1384377137,
  'retweet': [],
  'user_name': 'Angelus1701',
  'likes': [],
  'tweet_id': 400732874678087680,
  'user_id': 135092899},
 {'reply': [],
  'text': 'Breaking News - VIDEO: Winfrey: The Butler, racism and Obama: American broadcaster and actress O... http://bbc.in/1cp6nkD  - BBCBreaking',
  'created_at': 1384377137,
  'retweet': [],
  'user_name': 'lomeinlive',
  'likes': [],
  'tweet_id': 400732874074120192,
  'user_id': 976772936},
 {'reply': [],
  'text': 'VIDEO: Winfrey: The Butler, rac

In [20]:
tweet_data = []
pageSize=1

#//Page 1
fakenews_items = fakenews_datatable.find({}).limit(100);

for fake_newsItem in fakenews_items:
    print(f"{fake_newsItem['id']} ---> {fake_newsItem['text_content']['text']}")
    

ProtocolError: Message length (20611297) is larger than server max message size (4194304)

In [4]:
#fakenews_items = fakenews_datatable.find({}).limit(20);

#fakenews_datatable.aggregate( [ { "$unwind": "$tags" },  { "count": "$tweets" } ] ).limit(10)

items = fakenews_datatable.aggregate([{
  "$project": {
      "_id": "$_id",
      "id":"$id",
      "text_content":"$text_content.$text",
      "tweetCount": { "$size": "$tweets" }
  }
    
}]);

df = pd.DataFrame(list(items))
df.head()

,_id,id,tweetCount
0,5c3445ffe8a490580c567e3c,politifact14927,197
1,5c3445ffe8a490580c567e3d,politifact14856,116
2,5c3445ffe8a490580c567e3e,politifact14733,33
3,5c3445ffe8a490580c567e3f,politifact14693,4
4,5c3445ffe8a490580c567e40,politifact14362,25


In [5]:
df = df.sort_values(by='tweetCount',ascending=False)

In [6]:
df.head(10)

,_id,id,tweetCount
84,5c34465ae8a490580c567e7b,politifact15018,1141
9,5c3445ffe8a490580c567e46,politifact14259,975
35,5c34461499023e6294754497,politifact14839,963
63,5c34463de8a490580c567e71,politifact13589,827
38,5c34462699023e62947544ac,politifact15004,430
30,5c344615f839832ff86827f0,politifact13854,376
19,5c34461144925f59105fd04f,politifact14238,352
102,5c34466599023e62947544bb,politifact15334,333
5,5c3445ffe8a490580c567e41,politifact15201,329
103,5c34466599023e62947544bc,politifact15048,296


In [7]:
df2=df.head(10)
df2['news_title']=""
for index,row in df2.iterrows():
    df2.set_value(index,'news_title',fakenews_datatable.find_one({"id":row["id"]})['text_content']['title'])

df2.head()

C:\Users\chami\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\chami\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


,_id,id,tweetCount,news_title
84,5c34465ae8a490580c567e7b,politifact15018,1141,Smoking In Cars With Children Is Illegal Start...
9,5c3445ffe8a490580c567e46,politifact14259,975,Bumble Bee Foods charged after man cooked with...
35,5c34461499023e6294754497,politifact14839,963,Morgue employee cremated by mistake while taki...
63,5c34463de8a490580c567e71,politifact13589,827,Pope Francis To Followers: “Koran And Holy Bib...
38,5c34462699023e62947544ac,politifact15004,430,"NFL Lawyer, Who Claimed Super Bowl Is ‘Rigged’..."


In [8]:
df2.to_csv("top_tweeted.csv")

In [9]:
politifacts=fakenews_datatable.find_one({"id":"politifact15048"})
print(politifacts['text_content']['title'])

Trump Declares Globalist Soros As “National Security Threat”…Using Obama’s 2014 Executive Order


In [10]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [11]:
for tweet in politifacts['tweets']:

        twitter_data=api.get_status(tweet['tweet_id'])
        print(twitter_data.text)
        print(twitter_data.user.location)
        print(datetime.utcfromtimestamp(tweet['created_at']).strftime('%Y-%m-%d %H:%M:%S'))
   
        
    

Trump Declares Globalist Soros As "National Security Threat"...Using Obama's 2014 Executive Order
https://t.co/JKEYt8RnCu
Arrest &amp; prosecute
ΜΟΛΩΝ ΛΑΒΕ Texas
2017-01-30 19:24:12
Trump Declares Globalist Soros As "National Security Threat"...Using Obama's 2014 Executive Order https://t.co/RQ9ELtiByb

2017-01-30 20:37:53
Trump Declares Globalist Soros As “National Security Threat”…Using Obama’s 2014 Executive Order https://t.co/GeDay7EjYl
Port St. Lucie, FL
2017-01-30 20:39:46
Trump Declares Globalist Soros As “National Security Threat”…Using Obama’s 2014 Executive Order https://t.co/qRUxBBXKij
Hoover ,AL
2017-01-30 20:47:49
Trump Declares Globalist Soros As "National Security Threat"...Using Obama's 2014 Executive Order https://t.co/9IQapbndZM
abuja
2017-01-30 20:51:21
https://t.co/37QuX65oQ6
TRUMP DECLARES GLOBALIST SOROS AS “NATIONAL SECURITY THREAT”…USING OBAMA’S 2014 EXECUTIVE ORDER
Texas, USA
2017-01-30 21:17:22
Trump Declares Globalist Soros As "National Security Threat"...Usi

TweepError: [{'code': 144, 'message': 'No status found with that ID.'}]

In [12]:
df2
for index,row in df2.iterrows():
    print(row['id'])

politifact15018
politifact14259
politifact14839
politifact13589
politifact15004
politifact13854
politifact14238
politifact15334
politifact15201
politifact15048


In [ ]:
twitter_data_list = []

for index,row in df2.iterrows():
   
    politifacts=fakenews_datatable.find_one({"id":row["id"]})
    print(politifacts['text_content']['title'])
    
    for tweet in politifacts['tweets']:
        twitter_data_dic = {}
        try:
            twitter_data_dic['id']=row['id']
            twitter_data_dic['title']=politifacts['text_content']['title']
            
            twitter_data=api.get_status(tweet['tweet_id'])
            print(twitter_data.text)
            print(twitter_data.user.location)
            print(datetime.utcfromtimestamp(tweet['created_at']).strftime('%Y-%m-%d %H:%M:%S'))
            print(twitter_data.user.screen_name)
            twitter_data_dic['twitter_text']=twitter_data.text
            twitter_data_dic['twitter_location']=twitter_data.user.location
            twitter_data_dic['twitter_time']=datetime.utcfromtimestamp(tweet['created_at']).strftime('%Y-%m-%d %H:%M:%S')
            twitter_data_dic['twitter_user']=twitter_data.user.screen_name
            
            twitter_data_list.append(twitter_data_dic)
            db.twitter_data_combined.insert(twitter_data_dic)
       
        except tweepy.RateLimitError:
            print("rate limit exceeded")
            time.sleep(60 * 15)
            continue
        except:
            print("Unexpected error:")

    
    

Smoking In Cars With Children Is Illegal Starting January 30, 2018
Smoking In Cars With Children Is Illegal Starting  January 30, 2018 https://t.co/0GMGFKYUkG
World
2018-02-03 19:13:08
BestAmazingTwts


C:\Users\chami\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


Smoking In Cars With Children Is Illegal Starting January 30, 2018 https://t.co/InClWrusQB
Ohio
2018-02-04 14:23:14
Ohjobfairs
Smoking In Cars With Children Is Illegal Starting January 30, 2018 https://t.co/1Tcb1OZE3e
Louisiana, USA
2018-02-05 23:52:22
JECcupcake
Smoking In Cars With Children Is Illegal Starting January 30, 2018 https://t.co/sGZ3Gvfeim

2018-02-06 01:19:39
garmentsgrace
Smoking In Cars With Children Is Illegal Starting January 30, 2018 https://t.co/S5pmf9FhZs
Baton Rouge, LA
2018-02-06 01:28:08
ElijahFreeman2
Smoking In Cars With Children Is Illegal Starting January 30, 2018 https://t.co/JzzbQxZ0IF

2018-02-06 01:38:27
jkupdegraff426
Unexpected error:
Smoking In Cars With Children Is Illegal Starting January 30, 2018 https://t.co/DfuWzH3F0I via @LoveThisPic_com
Ohio
2018-02-06 02:52:22
elovebug7
Smoking In Cars With Children Is Illegal Starting January 30, 2018 https://t.co/psE0Qr8GoX via @LoveThisPic_com
Charlottesville, VA
2018-02-06 05:13:49
crazy4dmb
Smoking In Car

Smoking In Cars With Children Is Illegal Starting January 30, 2018 https://t.co/TY7eGdZFoH
Connecticut 
2018-02-06 21:56:40
mr_figueroa84
Smoking In Cars With Children Is Illegal Starting January 30, 2018 https://t.co/Yx5vV4ObFu
Beverly Hills, CA
2018-02-06 22:00:41
MYCAgnew
Smoking In Cars With Children Is Illegal Starting January 30, 2018 https://t.co/Ah4xckkVFd

2018-02-06 22:04:00
egwm03
Unexpected error:
Smoking In Cars With Children Is Illegal Starting January 30, 2018 https://t.co/n97EIS9j9Y
Kentucky
2018-02-06 22:09:27
suzie_burke
Smoking In Cars With Children Is Illegal Starting January 30, 2018 https://t.co/Yyi8KGwryw
Baton Rouge, LA
2018-02-06 22:12:13
TayDivinorum
Smoking In Cars With Children Is Illegal Starting January 30, 2018 https://t.co/8BNrAD03E7
Camille's Heart
2018-02-06 22:12:50
Trillest_Hippy
Smoking In Cars With Children Is Illegal Starting January 30, 2018 https://t.co/dn4dQQOom7
New Jersey & Philadelphia
2018-02-06 22:18:41
_michael__allen
Smoking In Cars With

Smoking In Cars With Children Is Illegal Starting January 30, 2018 https://t.co/pIFkgy5Oug
Fayetteville, GA
2018-02-07 04:12:15
successengagers
Smoking In Cars With Children Is Illegal Starting January 30, 2018 https://t.co/YwQnABzwVN
Holiday, FL
2018-02-07 04:12:26
pawsportunities
Smoking In Cars With Children Is Illegal Starting January 30, 2018 https://t.co/HeTc6t4LYx

2018-02-07 04:16:25
Gramme01Teresa
Smoking In Cars With Children Is Illegal Starting January 30, 2018 https://t.co/feXVKNipOh
norwalk, cali
2018-02-07 04:26:45
djserjah9
Smoking In Cars With Children Is Illegal Starting January 30, 2018 https://t.co/vRQkRcqwXz

2018-02-07 04:47:20
Wallhangers_Art
Smoking In Cars With Children Is Illegal Starting January 30, 2018 https://t.co/J1EYmxN6bI
west virginia
2018-02-07 04:55:59
MccardleNatasha
Unexpected error:
Smoking In Cars With Children Is Illegal Starting January 30, 2018 https://t.co/jPvemLyj9x
Greensboro, NC
2018-02-07 04:56:48
mjsparksb
Smoking In Cars With Children Is

...

Pope Francis To Followers: “Koran And Holy Bible Are The Same” http://t.co/DgWaOlB5L9
Calabar, Nigeria
2015-09-10 17:46:06
bassuquo
Pope Francis To Followers: “Koran And Holy Bible Are The Same” http://t.co/T5sQdz8UDb

2015-09-10 18:11:20
ANONRogermoore
Pope Francis To Followers: “Koran And Holy Bible Are The Same” http://t.co/9lpoRkooU9
MAINE
2015-09-10 18:11:38
benturner214
Pope Francis To Followers: “Koran And Holy Bible Are The Same” http://t.co/6dE2fvms1n
Nigeria
2015-09-10 18:22:51
foluking2004
Pope Francis To Followers: “Koran And Holy Bible Are The Same” http://t.co/tzc5mmSGdK
Hampshire and Surrey
2015-09-10 19:01:58
TheAnxietyKing
Pope Francis To Followers: “Koran And Holy Bible Are The Same”  http://t.co/STQIupEjrF
The Old Continent 
2015-09-10 19:03:55
DKurdistan
Pope Francis To Followers: “Koran And Holy Bible Are The Same” | JEWSNEWS http://t.co/j9edAYDXss
Waterlooville Hampshire UK
2015-09-10 19:06:47
Macira1
Pope Francis To Followers: “Koran And Holy Bible Are The Same” 

Pope Francis To Followers: “Koran And Holy Bible Are The Same” http://t.co/ISVfA1xLxL
South Wales. UK.
2015-09-21 14:03:52
puremessenger
Pope Francis To Followers: "Koran And Holy Bible Are The Same" http://t.co/hlpkdOCDPQ
Costa Smeralda, Sardinia, IT
2015-09-21 15:54:09
Edward_Kaul
Pope Francis To Followers: "Koran And Holy Bible Are The Same" http://t.co/GdZ4NyGrLw
Holland, MI
2015-09-22 00:31:53
E2WestOfficial
Pope Francis To Followers: “Koran And Holy Bible Are The Same” http://t.co/cuEE86zcCp
Dallas, Texas
2015-09-22 00:32:28
duffer47
Pope Francis To Followers: “Koran And Holy Bible Are The Same” http://t.co/lSJA9sRJKZ
Texas
2015-09-22 02:48:07
GypsyRVer
Pope Francis To Followers: “Koran And Holy Bible Are The Same” http://t.co/hKb8PwnZm4
Individually and collectively,
2015-09-22 04:17:49
angelqasem1
Pope Francis To Followers: “Koran And Holy Bible Are The Same” - See more at: http://t.co/MCdGjXNkRm
Kranj, Slovenija
2015-09-22 05:28:20
dreychee
Pope Francis To Followers: "Koran An

Pope Francis To Followers: "Koran And Holy Bible Are The Same" | National Report http://t.co/LgBCtqcH63 via @wpusta
Jasper Alabama
2015-09-29 04:21:56
antnyllee
Pope Francis To Followers: “Koran And Holy Bible Are The Same” http://t.co/zttz5iddv7
Philadelphia, PA 19103
2015-09-30 02:18:26
JoeMirarchi
@gerfingerpoken2 Pope Francis To Followers: “Koran And Holy Bible Are The Same”-Luk16:13 No servant can serve two masters:
I'mrighthere!
2015-09-30 05:41:14
JimDarcy2
Pope Francis To Followers: "Koran And Holy Bible Are The Same" http://t.co/aPSEZSMX2C
Belton, TX
2015-09-30 14:25:45
thelaunch511
Pope Francis To Followers: "Koran And Holy Bible Are The Same" http://t.co/HbywTx2rso

2015-09-30 17:34:29
DattaSudarshan
Pope Francis To Followers: "Koran And Holy Bible Are The Same" http://t.co/EbD0hnKqLI
Harrah, Oklahoma
2015-10-01 03:29:23
firgey35
Pope Francis To Followers: “Koran And Holy Bible Are The Same” http://t.co/liQ65wNY5u
DC
2015-10-02 02:02:23
gina_b1
Pope Francis To Followers: “Ko

Pope Francis To Followers: “Koran And Holy Bible Are The Same” https://t.co/hlwAV8OJxz
Portland, OR
2016-01-20 01:12:31
humatrouble
Pope Francis To Followers: “Koran And Holy Bible Are The Same” https://t.co/YEBj6LFrw5
USA
2016-01-20 02:19:35
mariaeke
Pope Francis To Followers: “Koran And Holy Bible Are The Same” https://t.co/DpRPg8pwWV
Biafra
2016-01-20 09:44:18
jamesclementb3
Pope Francis To Followers: “Koran And Holy Bible Are The Same” https://t.co/95AxW72hBX
Italy
2016-01-20 19:22:18
GreensoftwareIT
Pope Francis To Followers: “Koran And Holy Bible Are The Same” https://t.co/4APWUi4OqT
California
2016-01-21 09:18:05
Faith_iii
Pope Francis To Followers: “Koran And Holy Bible Are The Same” | WashingtonPost https://t.co/GB2p6u717x
 Kumardhubi, India
2016-01-28 07:18:48
ajasay
Pope Francis To Followers: "Koran And Holy Bible Are The Same" | National Report https://t.co/1HRJLi65v9 via @wpusta
Denver, CO
2016-02-06 00:55:11
crkienast
Pope Francis To Followers: "Koran And Holy Bible Are T

Pope Francis To Followers: "Koran And Holy Bible Are The Same" - World News Politics https://t.co/jpstmMaCj2
Delaware, USA
2016-02-20 22:01:12
3028976308
Pope Francis To Followers: "Koran And Holy Bible Are The Same" https://t.co/Iqxps5Berr
Baton Rouge, Louisiana
2016-02-20 22:03:23
ACTofGBR
I'm sorry, what??? Pope Francis To Followers:Koran And Holy Bible Are The Same. https://t.co/BRXi8sxxY9
Alabama 
2016-02-20 22:08:34
BethBanne2
ALL politicians are globalists. And now==&gt;Pope Francis To Followers: "Koran And Holy Bible Are The Same" #Trump2016  https://t.co/bXPG8YQwdp
Chicago, IL
2016-02-20 22:14:30
fazetti51
POPE FRANCIS TO FOLLOWERS: “KORAN AND HOLY BIBLE ARE THE SAME” - JUst like Karl Marx' Manifest is the same as... https://t.co/dY38zijuhU
Corpus Christi, TX
2016-02-20 22:18:17
twteaparty
Unexpected error:
Pope Francis To Followers: &amp;#8220;Koran And Holy Bible Are The Same&amp;#8221;: https://t.co/gDnOYBZwSC
ionia, mi
2016-02-20 23:22:20
davisrex58
#POPE #FRANCIS TO FOLLO

In [ ]:
combined_data = pd.DataFrame(twitter_data_list)
combined_data

In [ ]:
combined_data.to_json(orient='records')

In [ ]:
with open('data.json', 'w') as outfile:
    json.dump(combined_data.to_json(orient='records'), outfile)

In [ ]:
combined_data.to_csv("data.csv")